In [ ]:
%cd
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/MyDrive/Colab Notebooks/diploma_thesis
%ls

/root
Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/diploma_thesis
bbox_ann.csv                 extract_full_masks.ipynb
bbox_ann_train.csv           imagenet/
Conformer_small_patch16.pth  NOTEBOOK_TRANSCAM/
CutLER/                      plot_images_for_thesis.ipynb
dataset/                     polygon_masks.py
dataset_exp_2/               __pycache__/
dataset.ipynb                test_cutler.ipynb
dataset_v2.ipynb             test_cutler_train.ipynb
dataset_yolo/                test_cutler_v2.ipynb
divide_dataset.ipynb         TransCAM/
è«ñêê_2022/                  ultralytics/
EXP_2/                       yolov8.ipynb
explore_masks.ipynb


In [ ]:
!pip install supervisely

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import json
from glob import glob 
import json
from matplotlib import pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import Rectangle
from tqdm import tqdm
import shutil
from PIL import Image
import cv2

import base64
import io
import zlib
import numpy as np
import supervisely as slys
from polygon_masks import get_rle_mask, rleToMask, bitmap2rle, mask_2_base64

# 1. Transform plant parts dataset annotations for the TransCAM

In [ ]:
# Get paths to images and annotations
path2img = sorted(glob(os.path.join('dataset/images/train/', '*.jpg'))) 
path2ann = sorted(glob(os.path.join('dataset/labels_json/train/', '*.json'))) 
print(len(path2img), len(path2ann))

# Read image names
img_names = [];
for path in path2img:
  img_name = path.split('/')[-1].split('.')[0];
  img_names.append( img_name );

# Part classes
object_type = {"background": 0,
              "stem": 1,  
              "leaf": 2, 
              "fruit": 3,
              "flower": 4,
              "root": 5,
              "other": 6}


345 345


In [ ]:
# Read annotations from json
ann_in_img = [];
hw_in_img = [];
for ann in path2ann:
  with open(ann) as file:
      data = json.load(file)

      h = data['size']['height'];
      w = data['size']['width'];
      objects = data['objects'];

      # define amount of the part instances in the image
      plant_instance = set();
      for part in objects:
        plant_instance.add(part['classTitle'].lower())
      
      # extract bit map for each instance
      plant_data = {};
      plant_id = {};
      for instance in plant_instance:
        plant_data[instance] = [];
        plant_id[instance] = [];

        for part in objects:
          if( part['tags']!=[] ):
            if(instance == part['classTitle'].lower()):
              plant_data[instance].append( part['bitmap'])
              plant_id[instance].append( part['tags'][0]['name'] );
      
      ann_in_img.append(plant_data)
      hw_in_img.append([h,w])


# Extract annotations of the object parts
counter = 0;
csv_matrix = [];
for ann in tqdm(ann_in_img):

  for class_name in ann.keys():

      # WRITE CSV DATA FORMAT
      csv_row = [];
      csv_row = [img_names[counter], object_type[class_name.lower()] ];
      csv_matrix.append(csv_row);

  counter = counter + 1;


100%|██████████| 345/345 [00:00<00:00, 26315.95it/s]


In [ ]:
# Write object parts annotations
import csv

# open the file in the write mode
with open('dataset/bbox_part_train.csv', 'w') as file:

    # create the csv writer
    writer = csv.writer(file)

    # write a row to the csv file
    for row in csv_matrix:
      writer.writerow(row);



# 2. Transform full plants dataset annotations for the TransCAM

In [ ]:
# Prepare full plant annotations
path2test = 'è«ñêê_2022/Train and test/test/';
path2train = 'è«ñêê_2022/Train and test/train/';

test_dataset_name = os.listdir(path2test);
train_dataset_name = os.listdir(path2train)

print(test_dataset_name)
print(train_dataset_name)

img_name_cat = {};
for super_cat in train_dataset_name:
  path2cat_images = sorted(glob(os.path.join(path2train+super_cat+'/img/', '*.jpg')))
  img_name_in_cat = [];
  for path in path2cat_images:
    img_name_in_cat.append( path.split('/')[-1].split('.')[0] )
  img_name_cat[super_cat] = img_name_in_cat.copy();


['Cassava Leaf Disease', 'Corn or Maize', 'Fruit Plants', 'Herbarium', 'Plant Pathology', 'Tomato detection', 'Wild Edible Plants', 'flower_classification']
['Cassava Leaf Disease', 'Corn or Maize', 'Fruit Plants', 'Herbarium', 'Plant Pathology', 'Tomato detection', 'Wild Edible Plants', 'flower_classification']


In [ ]:
# Extract full plant annotations
super_cat_type = {train_dataset_name[x]: x for x in range(len(train_dataset_name))}
print(super_cat_type)

# Extract supercategory labels
csv_matrix = [];
for img_name in img_names:

  for super_cat in train_dataset_name:
    if(img_name in img_name_cat[super_cat]):
      csv_row = [];
      csv_row = [img_name, super_cat_type[super_cat]];

  # WRITE CSV DATA FORMAT
  csv_matrix.append(csv_row);


{'Cassava Leaf Disease': 0, 'Corn or Maize': 1, 'Fruit Plants': 2, 'Herbarium': 3, 'Plant Pathology': 4, 'Tomato detection': 5, 'Wild Edible Plants': 6, 'flower_classification': 7}


In [ ]:
import csv

# open the file in the write mode
with open('dataset/bbox_full_train.csv', 'w') as file:

    # create the csv writer
    writer = csv.writer(file)

    # write a row to the csv file
    for row in csv_matrix:
      writer.writerow(row);

